In [ ]:
library(dplyr)
library(stringr)
library(lubridate)
library(ggplot2)

options(repr.matrix.max.cols=30)
options(repr.plot.width=6, repr.plot.height=6)

Simple graphics in R
------------------

We've spent a lot of time looking at tables, looking at different kinds of objects that make life easier, looking at simple strategies for nosing around a data set. 

Initially, data analysis was not the sort of thing a self-respecting academic would do. You'd ask your graduate students to generate a graphs or run an analysis for you. They would deliver a stack of paper and you would interpret the results. Then, in the 1970's John Tukey transformed data analysis into an honest intellectual exercise. In his text "Exploratory Data Analysis" he formulated a style of analysis that really informs programs like R. To Tukey, data analysis was a cyclic process involving multiple views of a data set. From simple numerical summaries to more advanced graphical representations. The data analyst would cycle between her ideas about patterns and relationships in the data, and computations that validate or contradict these ideas. Hypothesis leads to computation leads to hypothesis and so on.

<img src="http://ecx.images-amazon.com/images/I/41QMTYHJ4QL.jpg">

To Tukey, the more views you had of the data, the better. In the early 70s he developed a tool for interactive data visualization that generated an unbounded number of views. To get a sense of what I mean by interactive exploration, you can try a more recent program [ggobi](http://www.ggobi.org/). 

The struggle to find structure in data is really what statistics is all about. It's hard, however, to see data in more than 3 dimensions (scatterplots using x, y and z, say) and so the hunt was on to find a framework for teasing out patterns. This could be done with formal models (like regression) or other kinds of graphical displays that  extend the tried and true techniques like scatterplots. In the early 1990s this led to something called conditional plots or trellis displays (lattice in R). We'll see these also.

Finally, novelty. There was a desire to formalize and then expose the process by which basic graphics were made. So let's not start with a scatterplot and see what we can do with it, but instead break it down to its constitutent parts and develop a new framework for graphics from the bottom up. This is the spirit of the Grammar of Graphics or ggplot(2) in R.

OK, too much chatter. Let's make some pictures. We are going to use a data set associated with a [simple story from the NYT about the hardest places to live in the United States.](http://www.nytimes.com/2014/06/26/upshot/where-are-the-hardest-places-to-live-in-the-us.html?_r=0) It is county-level data. Quoting The UpShot, it consists of "six data points for each county in the United States: education (percentage of residents with at least a bachelor’s degree), median household income, unemployment rate, disability rate, life expectancy and obesity. We then averaged each county’s relative rank in these categories to create an overall ranking." Here's the data.

In [ ]:
hardest <- read.csv("http://graphics8.nytimes.com/newsgraphics/2014/06/16/worst-places/d92796c59951a5cccb3ad6411f599ad302a5c4bd/unemployment.tsv",sep="\t",na.strings=c("No Data","#N/A"))
head(hardest)

In [ ]:
sample_n(hardest,10)

The data are formatted for display in the popup that appears when you hover over the New York Times' plot. It might be more useful to extract the state names separately. Here's how we pull out state using a regular explression and the `str_replace()` function in `stringr`.

In [ ]:
str_replace(hardest$County,".*, (.*)","\\1")

And we can add the column to data set. 

In [ ]:
hardest <- mutate(hardest,state=str_replace(County,".*, (.*)","\\1"))
head(hardest)

Now, we have seen in this class that we might want to join several data sets together. Here is something from the Census that maps states to regions of the country. Have a look.

In [ ]:
regions <- read.csv("https://www2.census.gov/programs-surveys/popest/datasets/2010-2015/state/asrh/scprc-est2015-18-pop-res.csv",na.strings="X",as.is=TRUE)
head(regions)

In [ ]:
sample_n(regions,10)

In [ ]:
count(regions,REGION)

Region 0 looks a little weird. What is that?

In [ ]:
filter(regions,REGION==0)

The NA also looks a little weird. What is that?

In [ ]:
filter(regions,is.na(REGION))

Finally, we join the two data sets, adding information about the region number to our hardest data set. We are keeping ust the state name and region number, but of course we could keep population or any other set of columns we might find useful later.

In [ ]:
head(select(regions,NAME,REGION))

In [ ]:
hardest <- left_join(hardest,select(regions,NAME,REGION),by = c("state"="NAME"))
dim(hardest)

In [ ]:
sample_n(hardest,10)

To make this more interpretable, we can "map the values" 1 through 4 to the words describing the regions. We then add this new column to the data set using mutate.

In [ ]:
hardest <- mutate(hardest,rname = c("Northeast","Midwest","South","West")[REGION])
sample_n(hardest,10)

**GGplot**

A relatively new, major graphics system (aside from special graphics that come along with new models, say) is `ggplot` -- well, `ggplot2.` It comes from Lee Wilkinson's book "Grammar of Graphics." In it, Wilkinson tried to come up with a system for describing any plot. It was meant to not only recreate existing graphical forms, but allow for the creation of new ones. 

`ggplot2` was created by, well it's getting old now but, Hadley Wickham. You can read [the formal documentation](http://docs.ggplot2.org/current/index.html). There is also [a great cheetsheat for ggplot2](https://www.rstudio.com/wp-content/uploads/2016/11/ggplot2-cheatsheet-2.1.pdf) and I recommend this [tutorial for ggplot2](http://zevross.com/blog/2014/08/04/beautiful-plotting-in-r-a-ggplot2-cheatsheet-3/). 

In [ ]:
hardest <- na.omit(hardest)

Let's start with a high-level plotting routine that comes with `ggplot2` -- it is called `qplot()` for quick plot. Here's a quick scatterplot followed by some elaborations.

In [ ]:
qplot(education,unemployment, data = hardest)

In [ ]:
qplot(education,unemployment, data = hardest, color = rname)

In [ ]:
qplot(education,unemployment, data = hardest, color = rname, size=income)

Aside from scatterplots, we can also create barplots and histograms by specifying other "geom"'s -- bindings between data and geometric objects. Here we use "bar" for a barplot. Also available are "boxplot" and "dotplot" and "line" and "smooth" and "point" (which we have already seen). Here is [a nice summary of qplot()](http://ggplot2.org/book/qplot.pdf).

In [ ]:
qplot(rname, data = hardest, geom = "bar")

In [ ]:
qplot(education,unemployment, data = hardest, geom=c("point","smooth"))

`qplot()` is short for "quick plot" and it tries to live up to its name. Let's take a second and work with the underlying `ggplot2` expressions so you get a sense of how this framework is constructed. (It would seem a little needless to give you another framework that looks like plot() -- what extra do you get by learning it?)

A plot begins with a map between data values and visual characteristics of our plot. 

In [ ]:
aes(x='obesity', y='life')

The command `ggplot()` takes a data set and constructs a plot using the aesthetics bindings and other information you provide. Here we get a set of x- and y-axes.

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))

The plot is blank because we have only set up the plot, made the bindings. Now we tell it what kind of geometric object we want to create. Points, lines, polygons...

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()

Notice that we added points by simply adding the specification for a geometric object. OK, let's break things down a little. The grammar of graphics builds on the following components of a plot.

* **data** in ggplot, data must be stored as a data frame
* **a coordinate system** describes 2-D space that data is projected onto (for example, Cartesian       coordinates, polar coordinates, map projections, and so on)
* **geoms** describe type of geometric objects that represent data (for example, points, lines,  polygons)
* **aesthetics** describe visual characteristics that represent data (for example, position, size,   color, shape, transparency, fill)
* **scales** for each aesthetic, describe how visual characteristic is converted to display values   (for example, log scales, color scales, size scales, shape scales, ...
* **stats** describe statistical transformations that typically summarize data (for example, counts, means, medians, regression lines)
* **facets** describe how data is split into subsets and displayed as multiple small graphs

geom_point() says that we want to render our x and y data as points. We can further adapt them by assigning colors and other features. 

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+
    geom_point()+
    coord_polar(theta="y")

You get the idea. We "add" components to a plot. Here we limit the y-range on the axis. A similar construction works for limiting the x-axis.

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()+ylim(72,76)

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()+scale_y_reverse()

We can add scales, adapting to the type of measurement we're plotting. Is it on the log-scale? Is it a date? Is it discrete? We can also add various statistical overlays to the plot...

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()+stat_smooth(method=lm,color="blue")

In [ ]:
ggplot(hardest,aes(x =education,y=unemployment))+geom_point()+stat_smooth(method=lm,color="blue")

In [ ]:
ggplot(hardest,aes(x =education,y=unemployment))+geom_point()+stat_smooth(method=loess,color="blue")

Here we add a lot of components to the graphic...

In [ ]:
ggplot(hardest,aes(x=education,y=unemployment))+
    geom_point(color='lightblue')+
    stat_smooth(method="loess",color="black")+
    ggtitle("Life expectancy and obesity rates")+
    xlab("Percentage with College Education")+
    ylab("Unemployment rate")

... and here's a geometric component that represents a histogram...

In [ ]:
ggplot(hardest,aes(x=obesity))+geom_histogram()

In [ ]:
ggplot(hardest,aes(x=obesity))+geom_histogram(binwidth=0.5)

... or a smoothed histogram....

In [ ]:
ggplot(hardest,aes(x=obesity))+geom_density()

... and using region names to give us different colors.

In [ ]:
ggplot(hardest,aes(x=obesity,color=rname))+geom_density()

The structure of "small multiples" offered by lattice is known here as faceting...

In [ ]:
ggplot(hardest,aes(x=obesity))+geom_density()+facet_wrap(~rname,scales='fixed')

And various plots we've seen but using geom's.

In [ ]:
ggplot(hardest,aes(x=rname))+geom_bar()

In [ ]:
ggplot(hardest,aes(x=obesity, y=life, color=rname)) + geom_point()

In [ ]:
ggplot(hardest,aes(x=obesity, y=life, color=rname)) + stat_smooth(method="loess",se=FALSE)

In [ ]:
ggplot(hardest,aes(x=obesity,y=life))+
    geom_point(color="lightblue")+
    stat_smooth(method="loess",color="black")+
    facet_wrap(~rname,scales='fixed')

In [ ]:
ggplot(hardest,aes(x=rname,y=life))+geom_boxplot()

**Maps**

Of course these data were presented originally as maps and we should end with that presentation. `ggplot2` has some built-in map data for places like countries and states and counties. Let's load up the state database...

In [ ]:
states <- map_data("state")
head(states)

... and plot it. it's really quite easy. We are creating polygon outlines from the lat-long coordinates in the `states` dataframe. We are then filling them according to region number, which will give us a lovely candy-colored country.

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)

ggplot(states,aes(x = long, y = lat, fill = region, group = group)) + 
  geom_polygon(color = "white") + 
  coord_fixed(1.3) +
  guides(fill=FALSE)

Now, to create a choropleth map that is shaded according to data values, let's return to the county level data, load up map coordinates for the counties...

In [ ]:
counties <- map_data("county")
head(counties)

In [ ]:
dim(counties)

... and then "join" in the data from `hardest`. To combine the two data sets, `hardest` and `counties` we need a "key" in common. In this case we'll just paste `subregion` and `region` together to match the original `County` column in `hardest`. We will also turn the `County` column into lowercase to match what's in the `counties` dataframe.

In [ ]:
counties <- mutate(counties,County=str_c(subregion,", ",region))
head(counties)

In [ ]:
hardest <- mutate(hardest,County=tolower(County))
head(hardest)

Now combine the two data sets.

In [ ]:
counties <- left_join(counties, hardest, by = "County")
head(counties)

In [ ]:
dim(counties)

Finally, let's have a look. Here we just use a default color for the counties...

In [ ]:
ggplot(counties,aes(x = long, y = lat, group = group)) + 
  geom_polygon(color = "white",lwd=0.1) + 
  coord_fixed(1.3) +
  guides(fill=FALSE)

... and here we shade according to `life`. You can pick any variable...

In [ ]:
ggplot(counties,aes(x = long, y = lat, group=group, fill= life)) + 
    geom_polygon(lwd=0.1) + 
    coord_fixed(1.3)

This [introduction to mapping in `ggplot2`](http://eriqande.github.io/rep-res-web/lectures/making-maps-with-R.html) is a reasonable introduction and describes how you would get different colors, mimicking, say, the NYT's color scheme with green and orange.

**Finishing up**

As I mentioned, faceting is an attempt to see more than one dimension at a time. There are other devices that attempt this as well, to varying degrees of sincerity. Tell me what a `pairs()` plot does...

In [ ]:
pairs(select(hardest,education,income,unemployment,disability,life,obesity),pch=19,cex=0.5)

We finish with one different example, that gives you plenty of room to try things out. It is a file of election results from 2016. 

In [ ]:
election <- read.csv("https://github.com/cocteau/D4D/raw/master/data/election.csv")
head(election)

We'll again use regions in the country and characterize the vote in each state in the region. I give this as an example of a plot you might envision, mock up on paper, and then implement in code. `ggplot2` should give you the ability to produce imaginative graphics.

In [ ]:
options(repr.plot.width=4, repr.plot.height=10)

party_colors <- c("#2E74C0","#CB454A")

ggplot(election,aes(x=r_points,y=reorder(state,r_points),color=party))+
    scale_color_manual(values=party_colors)+
    geom_vline(xintercept=0)+geom_point()+
    labs(x="Point margin",y="")+
    facet_wrap(~census,ncol=1,scales="free_y")+
    guides(color=FALSE)

In [ ]:
party_colors <- c("#2E74C0","#CB454A")

ggplot(election,aes(x=r_points,y=reorder(state,r_points),color=party))+
    scale_color_manual(values=party_colors)+
    xlim(-100,100)+
    geom_vline(xintercept=0)+geom_point()+
    labs(x="Point margin",y="")+
    facet_wrap(~census,ncol=1,scales="free_y")+
    guides(color=FALSE)



**The stories in data: A drill**

We spoke in class about situations in which a data set appears and you are asked to use it as a source for a story. Perhaps it is an end point, perhaps a starting point. This happened to me at the Marshall Project. In my case the data set(s) came from the Florida Department of Corrections (FDOC), and specifically their OBIS database. The FDOC describes it this way.

>The Florida Department of Corrections updates this information regularly, to ensure that it is complete and accurate; however this information can change quickly. Therefore, the information in this file may not reflect the true current location, status, release date, or other information regarding an inmate.
<br><br>
This database contains public record information on felony offenders sentenced to the Department of Corrections. This information only includes offenders sentenced to state prison or state supervision. Information contained herein includes current and prior offenses.
<br><br>
Offense types include related crimes such as attempts, conspiracies and solicitations to commit crimes. Information on offenders sentenced to county jail, county probation, or any other form of supervision is not contained. The information is derived from court records provided to the Department of Corrections and is made available as a public service to interested citizens. The Department of Corrections makes no guarantee as to the accuracy or completeness of the information contained herein. Any person who believes information provided is not accurate may contact the Department of Corrections. The Florida Department of Corrections is not responsible for misinterpretation or inaccurate reporting by entities or persons utilizing this information

The data consist of interlinked files (technically, a relational database) that describe Florida's inmate population since 1997. Here is a list of the tables and a basic description of each.

>Inmate_release_root : Basic inmate information on inmates released since 10-1-1997
<br>Inmate_release_aliases: Aliases for these released inmates
<br>Inmate_release_offenses_CPS: Current prison offenses for these released inmates
<br>Inmate_release_offenses_prpr: Prior prison offenses for these released inmates 
<br>Inmate_release_residence: Release plan address for these released inmates
<br>Inmate_release_detainers: Detainers for these released inmates
<br>Inmate_release_incarhist: Incarceration history for these released inmates
<br>Inmate_release_scarsmarks: Tattoos for these released inmates
<br>Inmate_active_root:  Basic inmate information on active inmates
<br>Inmate_active_aliases: Aliases for these active inmates
<br>Inmate_active_offenses_CPS: Current prison offenses for these active inmates
<br>Inmate_active_offenses_prpr: Prior prison offenses for these active inmates 
<br>Inmate_active_detainers: Detainers for these active inmates
<br>Inmate_active_incarhist: Incarceration history for these active inmates
<br>Inmate_active_scarsmarks: Tattoos for these active inmates
<br>Offender_root: Basic offender information on current community supervision offenders
<br>Offender_aliases: Aliases for offenders
<br>Offender_offenses_CCS: Current community supervision offenses for offenders
<br>Offender_residence: Most recent address (excluding confidential addresses) for offenders

You can find the collection of files at [this Dropbox site](https://www.dropbox.com/sh/fu8bjib6w5qd1zv/AABOvm6lDN7X8vPJBxjv0aJZa?dl=0). And you can [read about OBIS here](http://www.dc.state.fl.us/pub/obis_request.html). 

Your job is to come up with a brief story pitch based on these data. This is a monster of a datbase, but you can find a corner of it and dig in. Remember our high-level questions to get you started.

1. Who collected these data?


2. What were the motivations and incentives for the group that collected the data?


3. Why were they collected originally? 


4. What methodology was used to assemble the information?


5. Were you given the complete data set or just a portion?


6. How have the data been used previously? And were the "results" published? Peer reviewed? Did they appear in a good journal or lead to some valuable insights?


7. Were the data made public under some kind of publication standard? Did they adhere to that standard and what burden does this place on your use? 


8. What format are the data in? Are they a spreadsheet (CSV) or in XML or JSON? Is some speciality format used, and if so, do you have the software you need to analyze it? 

Then, dig in. What variables are present in each table. Read one into R and have look. Do you understand all the fields and what might you learn by looking at them, computing a statistic from them? What do you want to compute and how would you get there?

Some of the titles of the variables are opaque, but you can [see more about each through a search portal offered by the FDOC.](http://www.dc.state.fl.us/OffenderSearch/Search.aspx)

Since the data are on Dropbox and not GitHub, you have to download a file you are interested in and place it in the same folder as your notebook. Here we read in the Active prisoner's details.

In [ ]:
active <- read.csv("INMATE_ACTIVE_ROOT.csv")
head(active)

What information is here? What does it mean to look at it in the aggregate? Are there unique points? Clusters? How are you going to find a story here?

Thor and I will help you on your journey!